In [2]:
!pip install whois

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.0 MB/s eta 0:00:00


In [3]:
import hashlib
from googlesearch import search
from bs4 import BeautifulSoup
import whois
import requests

def check_email_pwned(email):
    hashed_email = hashlib.sha1(email.encode()).hexdigest().upper()
    prefix = hashed_email[:5]
    url = f"https://api.pwnedpasswords.com/range/{prefix}"
    response = requests.get(url)

    suffixes = [line.split(":") for line in response.text.splitlines()]
    for suffix, count in suffixes:
        if hashed_email.endswith(suffix):
            return f"The email '{email}' has been pwned {count} times."

    return f"The email '{email}' has not been found in any known data breaches."

def google_search(query):
    search_results = []
    for j in search(query, num=5, stop=5, pause=2):
        search_results.append(j)
    return search_results

def whois_lookup(domain):
    try:
        whois_info = whois.whois(domain)
        return whois_info
    except whois.parser.PywhoisError:
        return None

def scrape_zaubacorp(name):
    search_url = f"https://www.zaubacorp.com/companysearchresults/{name}.html"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    company_results = []

    for result in soup.find_all('div', class_='content_box'):
        company_name = result.find('a', class_='corporation-name').text.strip()
        company_results.append(company_name)

    return company_results

def search_user_info(name, phone, email, keywords):
    user_info = {
        "Name": name,
        "Phone": phone,
        "Email": email,
        "Keywords": keywords
    }

    results = {}

    # Check for data breaches using Have I Been Pwned
    email_result = check_email_pwned(email)
    results["Data Breaches"] = [email_result]

    # Perform a web search for user information
    for info_type, value in user_info.items():
        search_query = f"{info_type}: {value}"
        search_results = google_search(search_query)
        results[info_type] = search_results

    # Search for potential business or organization listings associated with the user's information
    business_results = scrape_zaubacorp(name)
    results["Business Listings"] = business_results

    # Perform WHOIS lookup for domains associated with the user's information
    for result in results.get("Email", []):
        if result.startswith("http"):
            domain = result.split("/")[2]
            whois_result = whois_lookup(domain)
            if whois_result:
                results.setdefault("Domains", []).append({domain: whois_result})

    return results

if __name__ == "__main__":
    name = input("Enter your name: ")
    phone = input("Enter your phone number: ")
    email = input("Enter your email address: ")
    keywords = input("Enter keywords related to you (comma-separated): ").split(',')

    results = search_user_info(name, phone, email, keywords)

    print("\nResults:")
    for info_type, info_results in results.items():
        print(f"\n{info_type}:")
        for result in info_results:
            if isinstance(result, dict):
                print("\nDomains:")
                for domain, whois_info in result.items():
                    print(f"Domain: {domain}")
                    print(f"WHOIS Information: {whois_info}")
            else:
                print(result)


Enter your name: ishwar
Enter your phone number: 7296838846
Enter your email address: ishwarsinghpanwar1122@gmail.com
Enter keywords related to you (comma-separated): ishwar singh panwar, trading

Results:

Data Breaches:
The email 'ishwarsinghpanwar1122@gmail.com' has not been found in any known data breaches.

Name:
https://www.babycenter.com/baby-names/details/ishwar-867715
https://en.wikipedia.org/wiki/Ishwar_(name)
https://www.prokerala.com/kids/baby-names/ishwar-2555.html
https://www.onlymyhealth.com/baby-names/meaning-of-name-ishwar-1427779498
https://www.babycentre.co.uk/babyname/1051759/ishwar

Phone:
https://t.me/s/wetradedaily?before=527

Email:

Keywords:
https://in.linkedin.com/in/ishwarsinghpanwar
https://www.linkedin.com/pub/dir/Ishwar/Singh+Panwar
https://contactout.com/Ishwar-Singh-Panwar-24651247
https://isacfoundation.org/ishwar-singh-panwar/
https://www.instagram.com/cyb1z/?hl=en

Business Listings:
